<a href="https://colab.research.google.com/github/sophielouie/beer-recommendation-system/blob/main/GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy import spatial
import statistics as stats
import math
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split
from networkx.algorithms.dag import topological_sort
import networkx as nx
import plotly.graph_objects as go
from node2vec import Node2Vec

In [2]:
#import Node2Vec and Hybrid Notebooks
import node2vec_module
from node2vec_module import node2vec

In [3]:
#defining buckets and shared attributes values for Node2Vec algorithm
BUCKETS, SHARED_ATTRIBUTES = 4, 7

we want to import functions into our gui file
- transport endpoint is not connected
- we do not understand google file explorer

In [4]:
!pip install node2vec

You should consider upgrading via the '/Applications/anaconda3/bin/python -m pip install --upgrade pip' command.


In [4]:
import node2vec as n
import hybridmodel as h

# app interface

In [5]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Dropdown
from ipywidgets import interact, interactive, fixed, interact_manual
import hybridmodel
from hybridmodel import clean_beer_reviews
import pandas as pd

In [ ]:
# paragraph describing each model

In [6]:
def clean_beer_reviews():
    # storing beer review dataset
    beer_reviews = pd.read_csv("beer_reviews.csv")
    # creating a unique identifier for each beer using brewery name and beer name
    beer_reviews['Unique Beer Name'] = beer_reviews['brewery_name'] + ' ' + beer_reviews['beer_name']
    # storing beer profile dataset
    beer_profile = pd.read_csv("beer_profile_and_ratings.csv")
    # columns to drop from beer reviews
    drop_cols = ['brewery_id', 'brewery_name',  'beer_name', 'beer_abv', 'beer_beerid']
    # dropping columns from beer reviews
    beer_reviews.drop(columns = drop_cols, inplace = True)
    # columns to drop from  beer profile
    drop_cols = ['Name', 'Style', 'Brewery', 'Description',
                 'Min IBU', 'Max IBU', 'Alcohol', 'review_aroma', 'review_appearance', 'review_palate',
                 'review_taste', 'review_overall', 'number_of_reviews']
    # dropping columns from beer profile
    beer_profile.drop(columns = drop_cols, inplace = True)
    # combining beer review and beer profile datasets to have profile of each beer attached to every review
    df_beer = pd.merge(beer_reviews, beer_profile, left_on = 'Unique Beer Name', right_on = 'Beer Name (Full)', how = 'inner')
    # isolating the numerical columns that need to be scaled
    need_scaling = df_beer.drop(columns = ['review_time', 'review_profilename', 'beer_style', 'Unique Beer Name', 'Beer Name (Full)'])
    # storing the informational portion of the dataset that does not need scaling
    informational = df_beer[['review_time', 'review_profilename', 'beer_style', 'Unique Beer Name', 'Beer Name (Full)']]
    # renaming beer name column
    informational.rename(columns = {'Beer Name (Full)': 'Beer Name'}, inplace = True)

    # scaling the data
    scaler = MinMaxScaler()
    scaler.fit(need_scaling)
    need_scaling = pd.DataFrame(scaler.transform(need_scaling), columns = need_scaling.columns)

    # recombining the informational data and scaled data
    df = pd.concat([informational, need_scaling], axis = 1)
    return df

In [7]:
BEER_DF = clean_beer_reviews()

C:\Users\timse\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [8]:
# make a fucntion that returns a list of beers
def make_beer_list():
    return BEER_DF["Beer Name"].unique()


# list of user, beer combination
def make_beer_user_list(user):
    return BEER_DF.loc[BEER_DF["review_profilename"] == user]["Beer Name"]

# list of users to choose from
def make_user_list():
    return BEER_DF['review_profilename'].unique()

In [ ]:
print(len(make_beer_list()))

In [9]:
# MDOEL TYPE DROPDOWN TO USER/BEER OPTIONS
def model_dropdown_eventhandler(change):
    determine(model_dropdown.value)

option_list = ["Hybrid", "Node2Vec"]

user = make_user_list()[:100]
beer = make_beer_list()

def determine(x):
    third_dropdown.layout.visibility = "hidden"
    sec_dropdown.index = None
    if x == "Hybrid":
        sec_dropdown.options = user
    else:
        sec_dropdown.options = beer
        
# HYBRID MODEL USER CHOISE TO CORRESPONDING BEER OPTIONS
def beer_dropdown_eventhandler(change):
    determine_beer(sec_dropdown.value)

def determine_beer(x):
    third_dropdown.index = None
    beer_user = make_beer_user_list(x)
    if len(beer_user) > 0:
        third_dropdown.options = beer_user
        third_dropdown.layout.visibility = "visible"
    else: 
        third_dropdown.layout.visibility = "hidden"

model_dropdown = Dropdown(description="Select a model", options=option_list,
            style = {'description_width': 'initial'}, value=None)
#This will be users for when hybrid is selected, beers for when Node2Vec is selected
sec_dropdown = Dropdown(description="Choose one",
            style = {'description_width': 'initial'})
model_dropdown.observe(model_dropdown_eventhandler, names='value')
#This will only appear when Hybrid is selected and contains beer list
third_dropdown = Dropdown(description="Choose one", layout = {"visibility" : "hidden"},
            style = {'description_width': 'initial'})
sec_dropdown.observe(beer_dropdown_eventhandler, names='value')
num_recs = widgets.IntSlider(
            value=5,
            min=0,
            max=10,
            step=1,
            description='Number of recommendations:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d',
            layout=widgets.Layout(width='50%', height='40px'),
            style = {'description_width': 'initial'}
        )

display(model_dropdown, sec_dropdown, third_dropdown, num_recs)

Dropdown(description='Select a model', options=('Hybrid', 'Node2Vec'), style=DescriptionStyle(description_widt…

Dropdown(description='Choose one', options=(), style=DescriptionStyle(description_width='initial'), value=None…

Dropdown(description='Choose one', layout=Layout(visibility='hidden'), options=(), style=DescriptionStyle(desc…

IntSlider(value=5, continuous_update=False, description='Number of recommendations:', layout=Layout(height='40…

In [13]:
import node2vec as n
import hybridmodel

In [14]:
btn = widgets.Button(description='Start')
display(btn)
def btn_eventhandler(obj):
    if model_dropdown.value == 'Hybrid':
        if sec_dropdown.value and third_dropdown.value and num_recs:
            print('running')
            print(sec_dropdown.value)
            print(third_dropdown.value)
            print(num_recs.value)
            print(h.hybrid_model(sec_dropdown.value, third_dropdown.value, num_recs.value))
    
    elif model_dropdown.value == 'Node2Vec':
        if sec_dropdown.value and num_recs:
            print('running')
            print(node2vec(BUCKETS, SHARED_ATTRIBUTES, sec_dropdown.value, num_recs.value))
    
btn.on_click(btn_eventhandler)

Button(description='Start', style=ButtonStyle())

running
fodeeoz
Amstel Brouwerij B. V. Amstel Light
5


C:\Users\timse\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\timse\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
import sys
!{sys.executable} -m pip insta